In [1]:
import pandas as pd
import numpy as np

In [2]:
!wget https://doi.org/10.1371/journal.pgen.1001134.s002 -q

In [3]:
CGI_mm9 = pd.read_excel("journal.pgen.1001134.s002", skiprows=6)
CGI_mm9 = CGI_mm9.rename(columns={"CpG density": "CpG_density"})

In [4]:
CGI_mm9.head()

,Chromosome,Start,End,ID,Strand,CpG observed,CpG observed/expected,G+C composition,Length,CpG_density,...,Ensembl,Ensembl TSS,ES_H3K4me3relaxed,Brain_H3K4me3relaxed,ES_RNAPII,Brain_RNAPII,SpBlM_MAP,SpBlF_MAP,SpCeM_MAP,SpCeF_MAP
0,chr1,3521604,3522101,CGI1,+,29,0.816525,0.693603,297,9.764310,...,1,0,0,0,0,0,1,1,1,1
1,chr1,3660434,3662858,CGI2,+,178,0.728255,0.663219,2224,8.003597,...,1,1,1,1,1,1,0,0,0,0
2,chr1,4342301,4342747,CGI3,+,11,0.503156,0.597561,246,4.471545,...,1,0,0,0,0,0,1,1,0,1
3,chr1,4481601,4483972,CGI4,+,171,0.756949,0.645325,2171,7.876555,...,1,0,1,0,1,1,0,0,0,0
4,chr1,4486744,4487898,CGI5,+,53,0.697292,0.564990,954,5.555556,...,1,1,0,0,0,0,0,0,0,0


In [5]:
CGI_mm9[["Chromosome", "Start", "End"]].to_csv("CAP-CGI_mm9.bed", sep="\t", header=None, index=False)

In [6]:
!$elias/seqtools/./liftOver CAP-CGI_mm9.bed $elias/seqtools/mm9ToMm10.over.chain ../mm10/CAP-CGI_mm10.bed unmapped

Reading liftover chains
Mapping coordinates


In [7]:
CGI_mm9.loc[CGI_mm9["CpG_density"] <= np.quantile(CGI_mm9["CpG_density"], 0.25), "quartile_CpGdensity"] = 1
CGI_mm9.loc[CGI_mm9["CpG_density"] > np.quantile(CGI_mm9["CpG_density"], 0.25), "quartile_CpGdensity"] = 2
CGI_mm9.loc[CGI_mm9["CpG_density"] > np.quantile(CGI_mm9["CpG_density"], 0.5), "quartile_CpGdensity"] = 3
CGI_mm9.loc[CGI_mm9["CpG_density"] > np.quantile(CGI_mm9["CpG_density"], 0.75), "quartile_CpGdensity"] = 4

In [8]:
for i in range(4):
    CGI_mm9.loc[CGI_mm9["quartile_CpGdensity"] == i+1, 
                 ["Chromosome", "Start", "End"]].to_csv(f"CAP-CGI_CpG_density_quartile_{i+1}_mm9.bed", sep="\t", header=None, index=False)

In [9]:
!for f in CAP-CGI_CpG_density_quartile_?_mm9.bed; do $elias/seqtools/./liftOver CAP-CGI_mm9.bed $elias/seqtools/mm9ToMm10.over.chain "${f/mm9/mm10}" unmapped; done
!mv CAP-CGI_CpG_density_quartile_?_mm10.bed ../mm10

Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
Reading liftover chains
Mapping coordinates
